# Compiles various statistics for submissions and reviews

* Get the reviews for all the papers

In [77]:
import openreview
import pandas as pd
import numpy as np
import re

venue_id = 'auai.org/UAI/2024/Conference'

cred_file = open('cred_info.txt', 'r')
client = openreview.api.OpenReviewClient(
    baseurl='https://api2.openreview.net',
    username=cred_file.readline().strip(),
    password=cred_file.readline().strip()
)

cred_file.close()

venue_group = client.get_group(venue_id)
under_review_id = venue_group.content['submission_venue_id']['value']
submissions = client.get_all_notes(content={'venueid': under_review_id})



In [ ]:
def extract_submission_number(url):
    # This regular expression looks for the word 'Submission' followed by one or more digits
    match = re.search(r'Submission(\d+)', url)
    if match:
        # The first group contains the digits after 'Submission'
        return match.group(1)
    else:
        return None
    

def extract_string_before_first_period_or_comma(text):
    # Use regex to split the text at the first period or comma found
    parts = re.split(r'[.,]', text, maxsplit=1)
    if parts:
        return parts[0].strip()  # Returns the part before the period or comma, stripping any extra whitespace
    return None  # Return None if there's no period or comma in the text



def extract_string_up_to_second_colon(text):
    # Split the text at all colons
    parts = text.split(':')
    # Check if there are enough parts to have two colons
    if len(parts) > 2:
        # Rejoin the first two parts with a colon, adding back the second colon that was removed in splitting
        return ':'.join(parts[:2]) 
    return None  # Return None if there aren't enough colons


table = []
cnt = 0 

for submission in submissions:
    print(cnt)
    reviews = client.get_notes(forum=submission.forum)
    AC_recomm = 'NA'
    AC_conf = 'NA'
    AC_indiff = 'NA'
    num_comments = len(reviews)
    submission_ID = 0
    reviewers = []
    for j in range(num_comments):
        try:
            Decision = reviews[j].content['decision']['value']
        except:
            do_nothing = 1
        try:
            AC_recomm = reviews[j].content['Recommendation']['value']
            AC_conf = reviews[j].content['confidence']['value']
            AC_indiff = reviews[j].content['Indifference']['value']
        except:
            do_nothing = 1
        try:
            if submission_ID == 0:
                for k in range(len(reviews[j].readers)):
                    temp = extract_submission_number(reviews[j].readers[k])
                    if temp != None:
                        submission_ID = temp
        except:
            do_nothing = 1
        try:
            reviewers.append((extract_string_up_to_second_colon(reviews[j].content['Q6_Overall_score']['value']), extract_string_before_first_period_or_comma(reviews[j].content['Q8_Confidence_in_your_score']['value'])))
        except:
            do_nothing = 1
    cnt = cnt + 1 

    row = {
        "Submission_ID": submission_ID,
        "Decision": Decision,
        "AC_Recommendation": AC_recomm,
        "AC_Confidence": AC_conf,
        "AC_Indifference": AC_indiff,
        "Reviewers_Scores_Confidence": reviewers,
        "Number_of_Comments": num_comments,
    }
    
    # Append the row dictionary to the table list
    table.append(row)
    
df = pd.DataFrame(table)

# Now you can easily export this to a CSV file
df.to_csv("submissions_details.csv", index=False)

In [84]:
import ast

# Function to parse the string representation of list of tuples
# and calculate the average of first and second entry in tuples.
def calculate_average_scores(confidence_string):
    # Use 'ast.literal_eval' to safely evaluate the string as a Python literal
    tuples = ast.literal_eval(confidence_string)
    
    # Initialize sums
    sum_scores = 0
    sum_confidence = 0
    
    # Loop through each tuple
    for score_confidence in tuples:
        # Extract and sum up the numeric part of the scores and confidence
        sum_scores += int(score_confidence[0].split(':')[0])
        sum_confidence += int(score_confidence[1].split(':')[0])
    
    # Calculate averages
    average_score = round(sum_scores / len(tuples),1)
    average_confidence = round(sum_confidence / len(tuples),1)
    
    # Return the averages as a tuple
    return (average_score, average_confidence)

# Function to add a new column with average score and confidence
def add_average_score_confidence_column(file_path):
    # Read the Excel file
    df = pd.read_excel(file_path)
    
    # Apply the 'calculate_average_scores' function to each entry in the 'Reviewers_Scores_Confidence' column
    df['ave_score_confidence'] = df['Reviewers_Scores_Confidence'].apply(calculate_average_scores)
    
    # Split the tuple into two separate columns for better readability and use in Excel
    df[['average_score', 'average_confidence']] = pd.DataFrame(df['ave_score_confidence'].tolist(), index=df.index)
    df.drop('ave_score_confidence', axis=1, inplace=True)  # Drop the combined column if not needed
    
    # Save the updated DataFrame back to a new Excel file
    df.to_excel('submissions_details_26_Apr.xlsx', index=False)

# Replace 'your_file.xlsx' with the path to your actual Excel file
add_average_score_confidence_column('submissions_details.xlsx')


In [ ]:
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# Load the data from an Excel file
df = pd.read_excel('submissions_details.xlsx', engine='openpyxl')

# Add a new column that flags rows where 'Decision' and 'AC_recommendation' are different
df['Different'] = df['Decision'] != df['AC_recommendation']

# Save the DataFrame back to an Excel file, keeping the index to reference rows
output_path = 'submissions_details.xlsx'
df.to_excel(output_path, index=True, engine='openpyxl')

# Load the workbook and the specific worksheet
wb = load_workbook(output_path)
ws = wb.active

# Define the highlight style
red_fill = PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid')

# Apply formatting to rows where 'Different' is True
for idx, row in enumerate(df['Different'], start=2):  # start=2 to account for the header row
    if row:
        for cell in ws[idx]:
            cell.fill = red_fill

# Save the changes to the Excel file
wb.save(output_path)
wb.close()

print("Excel file has been updated and saved to", output_path)


* Get the number of submitted reviews for each paper

In [ ]:
summary = {'number of reviews':{}}
for reviews_paper in all_reviews:
    summary['number of reviews'].setdefault(str(reviews_paper[-1].number),len(reviews_paper) - 1)  

